# Importing modules

In [2]:
import random
import pandas as pd
from datetime import datetime 
from pandas import ExcelWriter

In [3]:
df = pd.read_csv("Archive_Soldes_generated.csv",header=None,index_col=0)
df.columns=["Debtor","Date","Solde"]
print(df)
print(df.dtypes)
df['Date'] =  pd.to_datetime(df['Date'], format='%Y-%m-%d')
print(df.dtypes)
df.sort_values(["Debtor"],ascending=True, inplace=True) 
all_id = pd.unique(df["Debtor"].sort_values())

      Debtor        Date       Solde
0                                   
0       7027  2020-04-02 -1618683.72
1       7034  2020-04-02  -324202.24
2       7048  2020-04-02  -768574.76
3       7055  2020-04-02  -774744.80
4       7062  2020-04-02 -1044889.86
...      ...         ...         ...
6359  577576  2020-04-08  -284036.97
6360  577583  2020-04-08  -776878.53
6361  577590  2020-04-08  -171608.46
6362  577597  2020-04-08  -804559.82
6363  578227  2020-04-08  -535618.12

[6364 rows x 3 columns]
Debtor      int64
Date       object
Solde     float64
dtype: object
Debtor             int64
Date      datetime64[ns]
Solde            float64
dtype: object


# Checking which day was the highest debits recorded

In [4]:
df.groupby(['Date'])['Solde'].sum().sort_values(ascending=True)

Date
2020-04-14   -5.216604e+08
2020-04-07   -4.745471e+08
2020-04-02   -4.590935e+08
2020-04-13   -4.567768e+08
2020-04-08   -4.523038e+08
2020-04-09   -4.488149e+08
2020-04-06   -4.474621e+08
2020-04-10   -4.004684e+08
2020-04-03   -3.981003e+08
Name: Solde, dtype: float64

# Checking the highest debtors.

In [5]:
df.groupby(['Debtor'])['Solde'].sum().sort_values(ascending=True)

Debtor
448097   -90215718.40
371027   -42992280.95
448034   -34475683.81
315104   -27745291.75
91027    -26539037.03
             ...     
555302     -649542.90
567244     -443350.41
556324         -33.00
556513           0.00
399111      359927.27
Name: Solde, Length: 715, dtype: float64

# Extraction les soldes qui ont dépassé 5M par jour et calcul de leur récurrence et occurences successives 

In [7]:
#Extraire les soldes qui ont dépassé 5M par jour

res33 = {}
res5 = pd.DataFrame(columns = ["Debtor","Date","Solde","Succession"])
for i in all_id:
    recurr_totale = 0 # Compteur de cmb de période de succesion au total ?
    occ_succ = 0 # Compteur cmb de jours successifs ?
    dfIndividu = df.query('Debtor == {}'.format(i))
    dfIndividu.reset_index(drop=True, inplace=True)
    #print(dfIndividu)
    ancien = False
    for k in range(dfIndividu.shape[0]):
        if dfIndividu["Solde"].iloc[k] <= -5000000:
            occ_succ += 1
            dfResult = pd.DataFrame({"Debtor":[dfIndividu["Debtor"].iloc[k]],"Date":[dfIndividu["Date"].iloc[k]],"Solde":[dfIndividu["Solde"].iloc[k]],"Succession":[occ_succ]})
            res5 = pd.concat([res5,dfResult])
            nouveau = True
        else: #recurr_totale +=1 # cmb de fois et cmb de période dans son historique le cdf a succédé 5m
            occ_succ = 0 # jtrouve pas de succession, remise du compteur à 0
            nouveau = False

        if(ancien != nouveau and ancien==False):
            recurr_totale += 1
        ancien = nouveau
    res33[i] = recurr_totale
print("Done !")

Done !


# Saving the result in res33 dataframe

In [8]:
res5.reset_index(drop=True, inplace=True)
print(res5)
res5id = pd.unique(res5["Debtor"].sort_values())

for k in list(res33):
    if k not in res5id:
        res33.pop(k)
print()
print()
res33 = pd.DataFrame(res33.items(), columns=['Debtor', 'Récurrence'])
print(res33)

    Debtor       Date        Solde Succession
0    91027 2020-04-09  -5305018.61          1
1   371027 2020-04-03  -5047890.36          1
2   371027 2020-04-07  -5564987.79          1
3   371027 2020-04-06  -5228182.83          2
4   371027 2020-04-02  -5845301.71          1
5   448034 2020-04-08  -6301543.52          1
6   448034 2020-04-09  -7245355.90          2
7   448034 2020-04-07  -5841286.24          1
8   448034 2020-04-14  -8234415.68          2
9   448097 2020-04-07 -23904686.64          1
10  448097 2020-04-13  -6547683.06          1
11  448097 2020-04-08 -10963894.48          1
12  448097 2020-04-14 -38308961.01          1
13  546251 2020-04-02  -5284145.13          1
14  556240 2020-04-13  -6324317.99          1
15  556401 2020-04-14  -5176280.83          1


   Debtor  Récurrence
0   91027           1
1  371027           3
2  448034           2
3  448097           4
4  546251           1
5  556240           1
6  556401           1


# Extracting debits exceeding 3M per each 3 successive days, and computing their recurrence and occurrence 

In [9]:
listRes = []
res3 = {}
res3j = pd.DataFrame(columns = ["Debtor","Date","Solde","Occurrences sucessives"])
for i in all_id:
        recurr_totale = 0 # Compteur de cmb de période de succesion au total ?
        occ_succ = 0 # Compteur cmb de jours successifs ?
        cpt=0
        dfIndividu = df.query('Debtor == {}'.format(i))
        dfIndividu.reset_index(drop=True, inplace=True)
        ancien = False
        
        for index,rows in dfIndividu.iterrows():
            if rows['Solde'] <= -3000000:
                cpt += 1
                occ_succ += 1
                listTemp = [rows['Debtor'],rows['Date'],rows['Solde'],occ_succ]
                listRes.append(listTemp)
                nouveau = True
            if (rows['Solde'] > -3000000 or index == dfIndividu.shape[0]-1 ): #recurr_totale +=1 # cmb de fois et cmb de période dans son historique le␣cdf a succédé 5m
                if cpt>=3:
                    res3j = pd.concat([res3j, pd.DataFrame(listRes,columns = ["Debtor","Date","Solde","Occurrences sucessives"])])
                    recurr_totale += 1
                cpt=0
                occ_succ = 0 # jtrouve pas de succession, remise du compteur à 0
                nouveau = False
                listRes=[]

        res3[i] = recurr_totale

print("Done !")


Done !


# Saving the result in res3 dataframe

In [10]:
res3j.reset_index(drop=True, inplace=True)
print(res3j)
res5id = pd.unique(res3j["Debtor"].sort_values())
for k in list(res3):
    if k not in res5id:
        res3.pop(k)
print()
print()
res3 = pd.DataFrame(res3.items(), columns=['Debtor', 'Récurrence'])
print(res3)

    Debtor       Date       Solde Occurrences sucessives
0    91027 2020-04-10 -4855530.61                      1
1    91027 2020-04-09 -5305018.61                      2
2    91027 2020-04-14 -4189490.61                      3
3   189027 2020-04-03 -3248894.08                      1
4   189027 2020-04-06 -3179255.06                      2
5   189027 2020-04-08 -4497585.58                      3
6   315104 2020-04-09 -3461041.88                      1
7   315104 2020-04-13 -3952804.95                      2
8   315104 2020-04-02 -3518036.32                      3
9   371027 2020-04-03 -5047890.36                      1
10  371027 2020-04-10 -4396841.08                      2
11  371027 2020-04-07 -5564987.79                      3
12  371027 2020-04-06 -5228182.83                      4
13  371027 2020-04-14 -3584819.48                      5
14  371027 2020-04-02 -5845301.71                      6
15  371027 2020-04-09 -4380777.84                      7
16  371027 2020-04-08 -4147616.

# Saving all results in 4 Excel sheets. 

In [11]:
with ExcelWriter("Reporting Caisse.xlsx") as writer:
    start_row = 1
    res5.to_excel(writer, sheet_name=">5M", startrow=start_row, header = True)
    res33.to_excel(writer, sheet_name="Récurrences", startrow=start_row, header = True)
    res3j.to_excel(writer, sheet_name=">3M", startrow=start_row, header = True)
    res3.to_excel(writer, sheet_name="Récurrences 3J", startrow=start_row, header = True)
    writer.save()

/home/hassan/anaconda3/envs/myenv/lib/python3.9/site-packages/xlsxwriter/workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
